In [14]:
# Importing needed library
import pandas as pd
import numpy as np
import beta_glu
import pandascharm as pc
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import blosum as bl
from Bio.SubsMat.MatrixInfo import blosum62 as blosum
from Bio import AlignIO
from Bio import SeqIO
from Bio.Align.Applications import MuscleCommandline
from Bio.Align import AlignInfo
from scipy.fft import fft, ifft
import scipy.stats as stats
from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from pyaaisc import Aaindex
from Bio import AlignIO


In [33]:
df = pd.read_excel('../beta-glucosidase/data/KcatKm w_ Temperatures.xlsx')

In [34]:
output = 'pNP-Glc kcat/Km (1/smM)'
encoding = 'trigram'
aln = '../beta-glucosidase/data/kcatKm w_Temperatures.aln'

In [35]:
X,y,holder = beta_glu.encode_temp(encoding, output, df, aln, key = None)

In [36]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state = 42)

In [37]:
# XGBR tuning
def XGBR(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    
    param_grid = {'colsample_bytree': [0.8], 'gamma': [0.5], 'max_depth': [4], 'min_child_weight': [1], 'subsample': [0.6]}
    
    if jack == False:
        grid_search=GridSearchCV(xgb.XGBRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(xgb.XGBRegressor(), param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    XGB_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return XGB_pred, pred_training, param

In [38]:
XG_pred, XG_train, param = XGBR(X_train, y_train, X_val)

In [39]:
all_result = []

In [40]:
all_result.append([output,"XGB Regression",encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, XG_train)), np.sqrt(metrics.mean_squared_error(y_val, XG_pred)), r2_score(y_train, XG_train),r2_score(y_val, XG_pred),str(param), stats.pearsonr(y_val,XG_pred)]) 

In [41]:
all_result

[['pNP-Glc kcat/Km (1/smM)',
  'XGB Regression',
  'trigram',
  nan,
  122.43498076953814,
  386.7483633327977,
  0.9974088115189355,
  0.943651206971876,
  "{'colsample_bytree': 0.8, 'gamma': 0.5, 'max_depth': 4, 'min_child_weight': 1, 'subsample': 0.6}",
  (0.9784658495650626, 1.3652100231518566e-78)]]

In [11]:
a,b =beta_glu.ml_process(encoding, output, df, aln, temp = True, jack = False ,  key = None)

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:680: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.019e+08, tolerance: 1.992e+05 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in 

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.009e+07, tolerance: 1.920e+05
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.616e+07, tolerance: 1.366e+05
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.023e+07, tolerance: 1.495e+05
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.852e+06, tolerance: 2.041e+05
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.263e+06, tolerance: 1.366e+05
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.619e+06, tolerance: 1.992e+05
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.598e+06, tolerance: 2.041e+05
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.336e+06, tolerance: 1.920e+05
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.191e+06, tolerance: 1.992e+05
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.424e+06, tolerance: 1.495e+05
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

In [12]:
b

,Output,Algorithm,Encoding Method,Code,RMSE Training,RMSE Val,R^2 train,R^2 Val,Pearson r
0,pNP-Glc kcat/Km (1/smM),Linear Regression,trigram,NaN,1641.389385,5.262885e+15,0.446432,-4.382842e+24,0.008600
1,pNP-Glc kcat/Km (1/smM),LASSO Regression,trigram,NaN,1300.271884,1.498103e+03,0.652611,6.448667e-01,0.823209
2,pNP-Glc kcat/Km (1/smM),Decision Tree Regression,trigram,NaN,2.309988,4.800819e+02,0.999999,9.635298e-01,0.984804
3,pNP-Glc kcat/Km (1/smM),Random Forest Regression,trigram,NaN,277.457630,5.212305e+02,0.984182,9.570100e-01,0.986427
4,pNP-Glc kcat/Km (1/smM),Support Vector Machine Regression,trigram,NaN,2255.533225,2.550829e+03,-0.045313,-2.960499e-02,0.377202
5,pNP-Glc kcat/Km (1/smM),Neural Network Regression,trigram,NaN,1559.155506,1.945977e+03,0.500510,4.007840e-01,0.655639
6,pNP-Glc kcat/Km (1/smM),Elastic Net Regression,trigram,NaN,1325.797113,1.514308e+03,0.638838,6.371422e-01,0.838396
7,pNP-Glc kcat/Km (1/smM),XGB Regression,trigram,NaN,119.349635,4.585779e+02,0.997073,9.667238e-01,0.983452
